In [7]:
import os
import pickle
import argparse
import h5py

In [8]:
h = h5py.File('./data/datasets/products/train/data.h5py','r')['train']

In [10]:
for i in h.keys():
    print(i)

asin
cate
skipvec


In [11]:
h['asin'][:10]

array([b'0000143561.jpg', b'0000037214.jpg', b'0000032069.jpg',
       b'0000031909.jpg', b'0000032034.jpg', b'0000589012.jpg',
       b'0000401048.jpg', b'0001019880.jpg', b'0001048813.jpg',
       b'0001057170.jpg'], dtype='|S14')

In [12]:
h['cate'][:10]

array([[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
checkpoint_path = 'checkpoints/netG__epoch_25.pth'

In [ ]:
netG = NetG()
netG.load_state_dict(torch.load(checkpoint_path, map_location=lambda storage, loc: storage))

### train sample

### val sample

### test sample

In [18]:
import pickle

In [22]:
pickle.loads(open('./data/datasets/products/y_vocab.pickle', 'rb').read())

{'-1': 8,
 'All Beauty': 26,
 'All Electronics': 21,
 'Amazon Fashion': 32,
 'Appliances': 38,
 'Arts Crafts & Sewing': 19,
 'Automotive': 18,
 'Baby': 16,
 'Baby Products': 31,
 'Beauty': 17,
 'Books': 0,
 'Buy a Kindle': 45,
 'CDs & Vinyl': 5,
 'Camera & Photo': 27,
 'Car Electronics': 36,
 'Cell Phones & Accessories': 11,
 'Clothing Shoes & Jewelry': 2,
 'Collectible Coins': 46,
 'Collectibles & Fine Art': 34,
 'Computers': 20,
 'Digital Music': 33,
 'Electronics': 14,
 'Entertainment': 47,
 'Furniture & D&#233;cor': 43,
 'GPS & Navigation': 40,
 'Gift Cards': 37,
 'Grocery & Gourmet Food': 23,
 'Health & Personal Care': 10,
 'Home & Kitchen': 9,
 'Home Improvement': 30,
 'Industrial & Scientific': 25,
 'Kindle Store': 44,
 'Kitchen & Dining': 24,
 'Luxury Beauty': 41,
 'MP3 Players & Accessories': 35,
 'Magazine Subscriptions': 42,
 'Movies & TV': 1,
 'Musical Instruments': 6,
 'Office & School Supplies': 15,
 'Office Products': 12,
 'Patio Lawn & Garden': 28,
 'Pet Supplies': 22,


In [24]:
import gzip
g = gzip.open('./data/datasets/products/metadata.json.gz', 'r')

In [29]:
with gzip.open('./data/datasets/products/metadata.json.gz', 'r') as f:
    for i, l in enumerate(f):
        pass

In [30]:
i

9430087

In [31]:
import pandas as pd

In [ ]:
pd.read_csv('./')

In [33]:
48 * 9000000 * 4 /1024/1024/1024

1.6093254089355469

In [39]:
import pickle
list(enumerate(pickle.loads(open('./data/datasets/products/tmp/products_tv_0.pkl','rb').read())))

[(0, '0001048791.jpg'),
 (1, '0000143561.jpg'),
 (2, '0000037214.jpg'),
 (3, '0000032069.jpg'),
 (4, '0000031909.jpg'),
 (5, '0000032034.jpg'),
 (6, '0000589012.jpg'),
 (7, '0001048775.jpg'),
 (8, '0000031852.jpg'),
 (9, '0001048236.jpg'),
 (10, '0000401048.jpg'),
 (11, '0001019880.jpg'),
 (12, '0001048813.jpg'),
 (13, '0001148427.jpg'),
 (14, '0001057170.jpg'),
 (15, '0001047566.jpg'),
 (16, '0001053396.jpg'),
 (17, '0000913154.jpg'),
 (18, '0000032050.jpg'),
 (19, '0001072986.jpg'),
 (20, '0001053744.jpg'),
 (21, '0001055178.jpg'),
 (22, '0001064487.jpg'),
 (23, '0001042335.jpg'),
 (24, '0000202010.jpg'),
 (25, '0001057138.jpg'),
 (26, '000136118X.jpg'),
 (27, '0000000868.jpg'),
 (28, '0000695009.jpg'),
 (29, '0001053655.jpg'),
 (30, '0001360000.jpg'),
 (31, '0000038504.jpg'),
 (32, '0001203088.jpg'),
 (33, '0001061100.jpg'),
 (34, '000107461X.jpg'),
 (35, '0000143529.jpg'),
 (36, '0000477141.jpg'),
 (37, '0001437879.jpg'),
 (38, '0001035649.jpg'),
 (39, '0001381245.jpg'),
 (40, '000

In [1]:
import h5py

In [2]:
h = h5py.File('./data/datasets/products/train/data.h5py')['train']

In [3]:
for i in h.keys():
    print(i)

asin
cate
docvec


In [4]:
h['asin'][:10]

array([b'0321732960.jpg', b'0321735722.jpg', b'0657745316.jpg',
       b'1403796890.jpg', b'141278509X.jpg', b'1453060464.jpg',
       b'1577911989.jpg', b'1603112251.jpg', b'1613170416.jpg',
       b'1616830867.jpg'], dtype='|S14')

In [7]:
h['docvec'][:12].shape

(12, 100)

In [11]:
d = {"a":1, "b":2}

In [28]:
[cap for key, cap in d.items()]

[1, 2]

In [25]:
[j for sub in [('a', 1), ('b', 2)] for j in sub[1]]

TypeError: 'int' object is not iterable

In [22]:
import pandas as pd

In [23]:
df = pd.read_csv('./data/datasets/products/products.tsv',sep='\t',header=None)

In [24]:
df[1].value_counts()

Sports & Outdoors         215965
Grocery & Gourmet Food     78638
Computers                   2315
Name: 1, dtype: int64